In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=32, inter_op_parallelism_threads=32)))

from keras.applications import Xception
from keras import models
from keras import layers
from keras import optimizers

import numpy as np
import gc

"""
./format_data/train/
./format_data/val/
"""
train_dir = "./format_data/train/"
test_dir = "./format_data/val/"

feature_size = 299
batch_size = 5
middle_feature_size = (10,10,2048)
middle_sample_size = 3000

Using TensorFlow backend.


In [2]:
train_gen = ImageDataGenerator(rescale=1./255,rotation_range=180,horizontal_flip=True,vertical_flip=True)
test_gen = ImageDataGenerator(rescale=1./255)

train_gener = train_gen.flow_from_directory(train_dir,target_size=(feature_size,feature_size),batch_size=batch_size,shuffle = True,seed = 100,class_mode="categorical")
test_gener = test_gen.flow_from_directory(test_dir,target_size=(feature_size,feature_size),batch_size=batch_size,class_mode="categorical")

for data_batch,label_batch in train_gener:
    print(data_batch.shape)
    print(label_batch.shape)
    break

Found 1886 images belonging to 11 classes.
Found 110 images belonging to 11 classes.
(5, 299, 299, 3)
(5, 11)


In [3]:
X_model_base = Xception(weights=None,include_top=False,input_shape=(feature_size,feature_size,3))
X_model_base.load_weights("./xception_weights_notop.h5")
X_model_base.trainable = True

In [4]:
for layer in X_model_base.layers:
    if layer.name.startswith('block14'):
        layer.trainable = True
    else:
        layer.trainable = False

In [5]:
X_model_base.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 149, 149, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 149, 149, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [6]:
top_model = models.load_model('Xception_top.h5')
top_model.trainable = False

In [7]:
model = models.Sequential()
model.add(X_model_base)
model.add(layers.Flatten())
model.add(top_model)

gc.collect()

4

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 10, 10, 2048)      20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 204800)            0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 11)                209782539 
Total params: 230,644,019
Trainable params: 4,748,800
Non-trainable params: 225,895,219
_________________________________________________________________


In [9]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['acc'])

In [10]:
history = model.fit_generator(train_gener,steps_per_epoch=150,epochs=5,validation_data=test_gener,validation_steps=20)

Epoch 1/5
150/150 [==============================] - 351s - loss: 2.8205 - acc: 0.6187 - val_loss: 4.8622 - val_acc: 0.3200
Epoch 2/5
150/150 [==============================] - 301s - loss: 1.8145 - acc: 0.6053 - val_loss: 3.7120 - val_acc: 0.3300
Epoch 3/5
150/150 [==============================] - 305s - loss: 1.2264 - acc: 0.6400 - val_loss: 2.9562 - val_acc: 0.3500
Epoch 4/5
150/150 [==============================] - 308s - loss: 1.0133 - acc: 0.6893 - val_loss: 2.6774 - val_acc: 0.4100
Epoch 5/5
150/150 [==============================] - 310s - loss: 1.1630 - acc: 0.6960 - val_loss: 2.7846 - val_acc: 0.4100


In [11]:
model.save('Xception_middle.h5')

In [12]:
for layer in model.layers:
    if layer.name == 'sequential_1':
        layer.trainable = True

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 10, 10, 2048)      20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 204800)            0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 11)                209782539 
Total params: 230,644,019
Trainable params: 214,531,339
Non-trainable params: 16,112,680
_________________________________________________________________


In [14]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['acc'])

In [15]:
history = model.fit_generator(train_gener,steps_per_epoch=100,epochs=5,validation_data=test_gener,validation_steps=30)

Epoch 1/5
100/100 [==============================] - 396s - loss: 2.1440 - acc: 0.5460 - val_loss: 4.8902 - val_acc: 0.1867
Epoch 2/5
100/100 [==============================] - 363s - loss: 1.5920 - acc: 0.6200 - val_loss: 3.2617 - val_acc: 0.3067
Epoch 3/5
100/100 [==============================] - 358s - loss: 1.4295 - acc: 0.6380 - val_loss: 2.6851 - val_acc: 0.4267
Epoch 4/5
100/100 [==============================] - 357s - loss: 1.1079 - acc: 0.6820 - val_loss: 2.7264 - val_acc: 0.3467
Epoch 5/5
100/100 [==============================] - 355s - loss: 1.0263 - acc: 0.7100 - val_loss: 2.4927 - val_acc: 0.2933


In [16]:
model.save('Xception_base_final.h5')